In [1]:
from retraining_pipeline import *
import soundfile

In [2]:
data_loader = NumpyDataLoader()
feature_calculator = MFCCs()
pre_processor = TrainTestSplit()
trainer = SklearnTrainer()
model_runner = SVCRunner()
model_loader = ModelLoader()
model_comparison = BasicComparison()
model_saver = ModelSaver()

In [3]:
%cd /home/victor/Projects/doorbell_detector
data, labels = data_loader.run()

/home/victor/Projects/doorbell_detector


In [ ]:
data_val = data_loader.load_wav('deploy/data/unlabeled_data')
X_val = feature_calculator.run(data_val)
X_val.shape

In [42]:
print(data.shape)
print(labels.shape)
print(labels)
print(labels.sum())
np.where(np.all(data == data_val, axis=1))

(3330, 11025)
(3330,)
[1 1 1 ... 0 0 0]
180


(array([3280]),)

In [43]:
import soundfile as sf
from playsound import playsound
import time

for i in range(1):
    i = 3280
    print(labels[i])
    sf.write('temp.wav', data[i], 22050, 'PCM_24')
    playsound("temp.wav")
    time.sleep(2)

0


In [44]:
features = feature_calculator.run(data)

100%|██████████| 3330/3330 [00:18<00:00, 175.55it/s]


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, data_train, data_test \
     = train_test_split(features, labels, data, test_size=0.2,
                        random_state=42)

In [46]:
X_train.shape

(2664, 13)

In [49]:
np.where(np.all(X_train == X_val, axis=1))

(array([1241]),)

# Old Model

In [50]:
old_model = model_loader.run()

In [53]:
old_test_pred = old_model.predict(X_test)
wrong = np.where(old_test_pred != y_test)
wrong

(array([587]),)

In [54]:
sf.write('temp.wav', data_test[wrong[0][0]], 22050, 'PCM_24')
playsound("temp.wav")

In [57]:
old_train_pred = old_model.predict(X_train)
wrong = np.where(old_train_pred != y_train)
wrong

(array([1241]),)

In [49]:
old_model.predict(X_val)

array([1])

# New Model

In [58]:
new_model = trainer.run(X_train, X_test, y_train, y_test)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.991 total time=   0.1s
[CV 2/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.979 total time=   0.1s
[CV 3/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.981 total time=   0.1s
[CV 4/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.974 total time=   0.0s
[CV 5/5] END ......C=1, gamma=0.001, kernel=rbf;, score=0.985 total time=   0.0s
[CV 1/5] END .....C=1, gamma=0.0001, kernel=rbf;, score=0.983 total time=   0.0s
[CV 2/5] END .....C=1, gamma=0.0001, kernel=rbf;, score=0.977 total time=   0.0s
[CV 3/5] END .....C=1, gamma=0.0001, kernel=rbf;, score=0.976 total time=   0.0s
[CV 4/5] END .....C=1, gamma=0.0001, kernel=rbf;, score=0.979 total time=   0.0s
[CV 5/5] END .....C=1, gamma=0.0001, kernel=rbf;, score=0.983 total time=   0.0s
[CV 1/5] END .....C=10, gamma=0.001, kernel=rbf;, score=0.989 total time=   0.0s
[CV 2/5] END .....C=10, gamma=0.001, kernel=rbf;,

In [66]:
new_test_pred = new_model.predict(X_test)
wrong = np.where(new_test_pred != y_test)
wrong

(array([277, 587, 649]),)

In [68]:
for index in wrong[0]:
    print(index)
    sf.write('temp.wav', data_test[index], 22050, 'PCM_24')
    playsound("temp.wav")
    time.sleep(2)

277
587
649


In [69]:
new_train_pred = new_model.predict(X_train)
wrong = np.where(new_train_pred != y_train)
wrong

(array([ 195,  520,  716,  747,  879, 1163, 1202, 1757, 2070]),)

In [71]:
for index in wrong[0]:
    print(index)
    sf.write('temp.wav', data_train[index], 22050, 'PCM_24')
    playsound("temp.wav")
    time.sleep(2)

195
520
716
747
879
1163
1202
1757
2070
